In [ ]:
import random

import util
from test_framework.address import program_to_witness
from test_framework.key import ECKey, SECP256K1_ORDER, generate_key_pair, generate_schnorr_nonce
from test_framework.messages import CScriptWitness, CTxInWitness, sha256
from test_framework.musig import generate_musig_key, aggregate_schnorr_nonces, sign_musig, aggregate_musig_signatures
from test_framework.script import TaprootSignatureHash, SIGHASH_ALL_TAPROOT

# 2.2 TapTweak

* Tweaking the Public Key
* Commitment Schemes with Tweaks
* Spending a (tweaked) taproot output along the key path

The linear property of bip-schnorr means that we can encode a commitment into a public key, and then reveal that commitment when signing with the private key. We do that by _tweaking_ the private key with the commitment, and using the associated _tweaked_ pubkey. When signing, we can reveal that the original private key was tweaked by the commitment.

## Tweaking the Public Key

Instead of using our original public key as the witness program, we use a tweaked public key.

* `[01] [33B Tweaked Public Key]`

Tweaking a public key means to alter it with a value (the tweak) such that it remains spendable with knowledge of the original private key and tweak.

* `x` / `P`: Original private key / public key pair
* `t` / `T`: Tweak value / tweak point
* Output script: `[01] [P + T]` = `[01] [xG + tG]`
* Spendable by the tweaked private key: `x + t`

An observer cannot distinguish between a tweaked and untweaked public key.

#### Example 2.2.1: Signing with a tweaked keypair

In this example, we generate a key pair as before, and then tweak both the private key and public key. We then sign with the tweaked private key and verify that the signature is valid.

A _tweak_ is positive scalar value `t` where `0 < t < SECP256K1_ORDER`. There is an associated tweak point `T` such that `T = t*G`.

The private key is tweaked by the tweak scalar: `x' = x + t` and the public key is tweaked by the tweak point: `P' = P + T`.

The tweaked private key `x'` can be used to produce a valid signature for the tweaked pubkey `P'`.

In [ ]:
# Generate a key pair
privkey, pubkey = generate_key_pair()

print("Private key: {}\nPublic key: {}\n".format(privkey.secret, pubkey.get_bytes().hex()))

# Generate a random tweak 0 < t < SECP256K1_ORDER and its associated point
tweak = random.randrange(1, SECP256K1_ORDER)
tweak_private = ECKey().set(tweak, True)
tweak_point = tweak_private.get_pubkey()
print("Tweak scalar: {}\nTweak point: {}\n".format(tweak_private.secret, tweak_point.get_bytes().hex()))

# Derive the tweaked private key and public key
privkey_tweaked = privkey + tweak_private
pubkey_tweaked = pubkey + tweak_point
print("Tweaked private key: {}\nTweaked pubkey: {}\n".format(privkey_tweaked.secret, pubkey_tweaked.get_bytes().hex()))

# Sign the message with tweaked key pair and verify the signature
msg = sha256(b'msg')
sig = privkey_tweaked.sign_schnorr(msg)
assert pubkey_tweaked.verify_schnorr(sig, msg)
print("Success!")

#### _Programming Exercise 2.2.2:_  Signing with a tweaked 2-of-2 MuSig key pair

In this exercise, we tweak an MuSig aggregate pubkey, and then sign for it using the individual participant keys. The MuSig pubkey aggregation step is done for you.

_Question: Which participant(s) need to tweak their private keys?_

In [ ]:
# Generate key pairs
privkey1, pubkey1 = generate_key_pair()
privkey2, pubkey2 = generate_key_pair()

# Create an aggregate MuSig pubkey
c_map, agg_pubkey = generate_musig_key([pubkey1, pubkey2])

# Apply challenge factors to keys
privkey1_c = privkey1.mul(c_map[pubkey1])
privkey2_c = privkey2.mul(c_map[pubkey2])
pubkey1_c = pubkey1.mul(c_map[pubkey1])
pubkey2_c = pubkey2.mul(c_map[pubkey2])

# Tweak musig public key
# Method: ECPubKey.tweak_add()
tweak = random.randrange(1, SECP256K1_ORDER)
agg_pubkey_tweaked =  # TODO: implement

# Nonce generation & aggregation
# Method: aggregate_schnorr_nonces()
# Remember to negate the individual nonce values if required
R_agg, negated =  # TODO: implement
if negated:
    # TODO: implement

# Signing and signature aggregation
msg = sha256(b'msg')

# Sign individually and then aggregate signatures
# Method: sign_musig(private_key, nonce_key, nonce_point, public_key, msg)
# Method: aggregate_musig_signatures(partial_signature list)
sig_agg =  # TODO: implement

assert agg_pubkey_tweaked.verify_schnorr(sig_agg, msg)
print("Success!")

## Commitment schemes with tweaks

Taproot uses the tweak as a commitment for spending script paths. However, simply applying the committed value as a public key tweak is not sufficient, as this does not represent a secure cryptographic commitment.

![test](images/taptweak0.jpg)

Instead, the committed value must first be hashed with the untweaked public key point. **This prevents modification of both untweaked secret and tweak for a given tweaked pubkey point Q.**

#### Example 2.2.3: modifying the tweak for a tweaked public key Q

In this example we demonstrate an insecure commitment scheme. Simply tweaking the private key with a value `c` allows the pubkey equation `Q = x'G + c'G` to be solved for any `c'` by modifying `x'`.

In [ ]:
# Generate a key pair
x, P = generate_key_pair()
print("Private key: {}\nPublic key: {}\n".format(x.secret, P.get_bytes().hex()))

# Tweak the public key
t = random.randrange(1, SECP256K1_ORDER)
print("Tweak: {}".format(t))
Q = P.tweak_add(t)

# Create a fake tweak
t2 = random.randrange(1, SECP256K1_ORDER)
print("Tweak 2: {}\n".format(t2))

# Solve: x` = x - t' + t
x_int = x.as_int()
x2_int = (x_int - t2 + t) % SECP256K1_ORDER

x2_key = x = ECKey().set(x2_int, True)
P2 = x2_key.get_pubkey()
Q2 = P2.tweak_add(t2)

print("Tweaked pubkey for x tweaked by t: {}".format(Q.get_bytes().hex()))
print("Tweaked pubkey for x2 tweaked by t2: {}".format(Q2.get_bytes().hex()))

#### Example 2.2.4 - Tweaking the pubkey with `H(P|msg)`

In this example, we demonstrate a _secure_ commitment scheme. The private key is tweaked with the scalar `H(P|c)`. Since `P` appears both inside and outside the hash, it isn't possible to solve for a different `c` by modifying `x'`.

In [ ]:
# Key pair generation
privkey1, pubkey1 = generate_key_pair()
print("Private key: {}\nPublic key: {}\n".format(x.secret, pubkey.get_bytes().hex()))

# Compute the tweak from H(P|msg)
commitment = b'commitment'
ss = sha256(pubkey.get_bytes())
ss += sha256(commitment)
t = sha256(ss)

# Determine tweak point
tweak = ECKey().set(t, True)
tweak_point = tweak.get_pubkey()
print("Tweak scalar: {}\nTweak point: {}\n".format(tweak.secret, tweak_point.get_bytes().hex()))

privkey_tweaked = privkey + tweak
pubkey_tweaked = pubkey + tweak_point

# Sign message and verify signature
msg = sha256(b'msg')
sig = privkey_tweaked.sign_schnorr(msg)

assert pubkey_tweaked.verify_schnorr(sig, msg)
print("Success!")

## Spending a taproot output along the key path

In this exercise, we'll create a segwit version 1 output that sends to a tweaked public key. We'll them spend that output along the key path using the tweaked private key.

Such as spend does not reveal the committed tweak to the observer and is indistinguishable any other key path spend.

#### _Programming Exercise 2.2.5:_ Construct taproot output with tweaked public key

In [ ]:
# Example key pair
privkey = ECKey().set(102118636618570133408735518698955378316807974995033705330357303547139065928052, True)
internal_pubkey = privkey.get_pubkey()

# Example tweak
taptweak = bytes.fromhex('2a2fb476ec9962f262ff358800db0e7364287340db73e5e48db36d1c9f374e30')

# Tweak the public key
# Method: use tweak_add()
taproot_pubkey =  # TODO: implement
taproot_pubkey_b =  # TODO: implement

# Derive the bech32 address
# Tip: set the first byte of taproot_pubkey to 0 or 1 and then call program_to_witness(version_int, pubkey_bytes)
address =  # TODO: implement

assert address == "bcrt1pq9lku0vuddzvcte8yvt3xct0dk6cjqeq2yzqp3vwpvh2e8afqpvqqyftl09"
print("Success! Address: {}".format(address))

#### Example 2.2.6: Start Bitcoin Core node and send coins to the taproot address

Only run setup once, or after a clean shutdown.

In [ ]:
# Start node
test = util.TestWrapper()
test.setup()
node = test.nodes[0]

# Generate coins and create an output
tx = node.generate_and_send_coins(address)
print("Transaction {}, output 0\nsent to {}\n".format(tx.hash, address))

#### Example 2.2.7: Construct `CTransaction` and populate inputs

We use the `create_spending_transaction(node, txid)` convenience function.

In [ ]:
# Create a spending transaction
spending_tx = test.create_spending_transaction(tx.hash)
print("Spending transaction:\n{}".format(spending_tx))

#### _Programming Exercise 2.2.8:_ Spend taproot output with key path

In [ ]:
# Sign transaction with tweaked private key
# Method: TaprootSignatureHash(tx, output_list, hash_type=int, input_index=int, scriptpath=bool)
sighash =  # TODO: implement
sig =  # TODO: implement

# Construct transaction witness
witness = CScriptWitness()
witness.stack.append(  # TODO: implement
witness_in = CTxInWitness()
witness_in.scriptWitness = witness
spending_tx.wit.vtxinwit.append(witness_in)

# Test mempool acceptance
assert node.test_transaction(spending_tx)
print("Success!")

#### _Shutdown TestWrapper_

In [ ]:
# Shutdown
test.shutdown()

**Congratulations!** In this chapter, you have:

- Learned how to tweak a public/private key pair with a value.
- Created an _insecure_ commitment scheme (by tweaking the keys with the raw commitment value) and a _secure_ commitment scheme (by tweaking with a hash of the commitment and the public key).
- Sent coins to a segwit v1 output with a tweaked public key, and later spent that output by signing with the tweaked private key.